- fhs_flashsale_product: (Thông tin flashsale của product)
    * period_id : id của flashsale
    * product_id : id của product 
    * original_price : giá gốc
    * flashsale_price : giá sale
    * total_items : số lượng sale
    
- fhs_flashsale_period: (Chứa thông tin flashsale về thời gian)
    * id: id của flashsale (== period_id của fhs_flashsale_product)
    * start_date, end_date: time bắt đầu kết thúc sale 
    * các cột khác không quan tâm
    
- fhs_catalog_product_entity: (Thông tin discount và ngày bắt đầu bán)
    * entity_id: id của product (==product_id trong fhs_flashsale_product)
    * sku : là mã sản phẩm (duy nhất và là khóa của sản phẩm dùng để truy các thông tin khác về sản phẩm)
    * created_at : ngày bắt đầu bán
    * discount_percent : phần trăm giảm giá so với giá bìa (trừ flashsale còn giá sản phẩm mọi lúc sẽ là giá gốc trừ đi disount này)

- fhs_cataloginventory_stock_item: Bãng này là số lượng sản phẩm tồn nhưng ko dùng được, do bên db bị mất dữ liệu
    
- fhs_sales_flat_order: Chứa thông tin đơn hàng
    * entity_id : id của đơn hàng (==order_id trong fhs_sales_flat_order_item)
    * state : trạng thái đơn hàng
    * status : trạng thái của trạng thái đơn hàng (ví dụ đơn hàng ở state đang giao, status là complete nghĩa là ở trang thái đang * giao đã hoàn thành)
    * created_at : ngày tạo đơn
    * updated_at : ngày cập nhật trạng thái
    * store_id : id của kho, id < 4 cho thương mại điện tử  (nhớ thêm điều kiện store_id<4 khi query bảng này)

- fhs_sales_flat_order_item: Chưa thông tin từng sản phẩm trong đơn hàng
    * order_id: id của đơn hàng (==entity_id trong fhs_sales_flat_order)
    * created_at: Ngày tạo đơn con
    * product_id, sku : id và mã sản phẩm

- fhs_catalog_product_entity_decimal: Bảng giá sản phẩm
    * entity_id: id của product (==entity_id trong fhs_catalog_product_entity)
    * attribute_id : loại giá (giá gốc 75, giá bán 76 (hoặc ngược lại chị không nhớ, chỉ quan tâm 2 loại này))
    * value : giá bán của sản phẩm với attribute tương ứng
    
- fhs_product_stock_log: Bảng này là thông tin các sản phẩm đang bán hiện tại trên thị trường
    * product_id, sku : id và mã của sp
    * qty: số lượng trong kho
    * is_in_stock: 1 là còn bán
    * created_at: không rõ
    * bảng này cũng không đáng tin lắm 
    
- product_dim : Thông tin sp
    * sku: mã
    * name: tên
    * cat1 tới cat5: là categories của sp
    * price : giá gốc

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
fhs_flashsale_product = pd.read_csv('../Data/Data_Fahasa/fhs_flashsale_product.csv')
fhs_flashsale_period = pd.read_csv('../Data/Data_Fahasa/fhs_flashsale_period.csv')
fhs_catalog_product_entity = pd.read_csv('../Data/Data_Fahasa/fhs_catalog_product_entity.csv')
fhs_sales_flat_order = pd.read_csv('../Data/Data_Fahasa/fhs_sales_flat_order.csv')
fhs_sales_flat_order_item = pd.read_csv('../Data/Data_Fahasa/fhs_sales_flat_order_item.csv')
fhs_catalog_product_entity_decimal = pd.read_csv('../Data/Data_Fahasa/fhs_catalog_product_entity_decimal.csv')
fhs_product_stock_log = pd.read_csv('../Data/Data_Fahasa/fhs_product_stock_log.csv')
product_dim = pd.read_csv('../Data/Data_Fahasa/product_dim.csv')

### Ta đầu tiên đổi tên các cột trong từng dataframe để tránh trùng lập khi mở rộng

In [3]:
def rename_columns(dataframe, dataframe_name):
    """
    rename columns with patern: dataframe_name.columns_name
    Input:
        dataframe: Dataframe Pandas
        dataframe_name: string
    """
    columns = dataframe.columns
    # rename columns
    new_names = []
    for index in range(len(columns)):
        new_name = f'{dataframe_name}.{columns[index]}'
        new_names.append(new_name)
    
    dataframe.columns = new_names

rename_columns(fhs_flashsale_product, 'fhs_flashsale_product')
rename_columns(fhs_flashsale_period, 'fhs_flashsale_period')
rename_columns(fhs_catalog_product_entity, 'fhs_catalog_product_entity')
rename_columns(fhs_sales_flat_order, 'fhs_sales_flat_order')
rename_columns(fhs_sales_flat_order_item, 'fhs_sales_flat_order_item')
rename_columns(fhs_catalog_product_entity_decimal, 'fhs_catalog_product_entity_decimal')
rename_columns(fhs_product_stock_log, 'fhs_product_stock_log')
rename_columns(product_dim, 'product_dim')

### Xóa các cột không cần thiết của fhs_flashsale_period

In [4]:
temp = fhs_flashsale_period.loc[:,
    ['fhs_flashsale_period.id', 'fhs_flashsale_period.start_date', 'fhs_flashsale_period.end_date']]
# xóa đi để tiết kiệm bộ nhớ
del fhs_flashsale_period
fhs_flashsale_period = temp

### Chuyển các chuỗi thời gian về định dạng datetime

In [5]:
def string_to_datetime(dataframe, column_name):
          dataframe[f'{column_name}'] \
          = pd.to_datetime(dataframe[f'{column_name}'],
                              format = '%Y/%m/%d %H:%M:%S')

string_to_datetime(fhs_flashsale_period, 'fhs_flashsale_period.start_date')
string_to_datetime(fhs_flashsale_period, 'fhs_flashsale_period.end_date')
string_to_datetime(fhs_catalog_product_entity, 'fhs_catalog_product_entity.created_at')
string_to_datetime(fhs_sales_flat_order, 'fhs_sales_flat_order.created_at')
string_to_datetime(fhs_sales_flat_order, 'fhs_sales_flat_order.updated_at')
string_to_datetime(fhs_sales_flat_order_item, 'fhs_sales_flat_order_item.created_at')
string_to_datetime(fhs_product_stock_log, 'fhs_product_stock_log.created_at')



In [6]:
print('fhs_flashsale_product', fhs_flashsale_product['fhs_flashsale_product.product_id'].max())
print('fhs_catalog_product_entity',fhs_catalog_product_entity['fhs_catalog_product_entity.entity_id'].max())
print('fhs_catalog_product_entity',fhs_catalog_product_entity['fhs_catalog_product_entity.sku'].count())
print('fhs_product_stock_log',fhs_product_stock_log['fhs_product_stock_log.sku'].count())
print('fhs_catalog_product_entity',fhs_catalog_product_entity['fhs_catalog_product_entity.sku'].count())
print('fhs_sales_flat_order_item',fhs_sales_flat_order_item['fhs_sales_flat_order_item.sku'].count())


fhs_flashsale_product 397207
fhs_catalog_product_entity 397396
fhs_catalog_product_entity 352880
fhs_product_stock_log 10250978
fhs_catalog_product_entity 352880
fhs_sales_flat_order_item 17828284


- Đầu tiên hãy rút ra một bảng gồm các đơn đặt hàng trong flash sale
    * Lưu ý các đơn đặt hàng này phải thành công
    * Sử dụng numpy để  query ra các dòng và lọc ra

In [7]:
condition1 = np.where((fhs_sales_flat_order['fhs_sales_flat_order.state'] == 'complete')
                        & (fhs_sales_flat_order['fhs_sales_flat_order.status'] == 'complete'))
fhs_sales_flat_order_complete = fhs_sales_flat_order.loc[condition1]

### Tiếp theo merge với fhs_sales_flat_order_item để có thông tin về tất cả các đơn đặt hàng

biết được đơn hàng đang mua những mặt hàng nào từ đó ta mới có thể biết được liệu mặt hàng đó có trong flashsale hay không, và phải nằm trong khoảng thời gian sale

In [8]:
fhs_sales_flat_order_complete_item  \
        = pd.merge(fhs_sales_flat_order_complete, fhs_sales_flat_order_item,
                        left_on = 'fhs_sales_flat_order.entity_id', 
                        right_on = 'fhs_sales_flat_order_item.order_id')
fhs_sales_flat_order_complete_item.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128362 entries, 0 to 11128361
Data columns (total 12 columns):
 #   Column                                Dtype         
---  ------                                -----         
 0   fhs_sales_flat_order.entity_id        int64         
 1   fhs_sales_flat_order.increment_id     float64       
 2   fhs_sales_flat_order.state            object        
 3   fhs_sales_flat_order.status           object        
 4   fhs_sales_flat_order.created_at       datetime64[ns]
 5   fhs_sales_flat_order.updated_at       datetime64[ns]
 6   fhs_sales_flat_order.store_id         int64         
 7   fhs_sales_flat_order_item.item_id     int64         
 8   fhs_sales_flat_order_item.order_id    int64         
 9   fhs_sales_flat_order_item.created_at  datetime64[ns]
 10  fhs_sales_flat_order_item.product_id  float64       
 11  fhs_sales_flat_order_item.sku         object        
dtypes: datetime64[ns](3), float64(2), int64(4), object(3)
memory usage: 

### Tiếp theo merge các cột flashsale để có thông tin về sản phẩm được bán trong đợt flashsale này

In [9]:
temp = pd.merge(fhs_flashsale_product, fhs_flashsale_period, 
          left_on='fhs_flashsale_product.period_id', right_on='fhs_flashsale_period.id')
fhs_flashsale_product_info = pd.merge(temp, fhs_catalog_product_entity, 
          left_on='fhs_flashsale_product.product_id', right_on='fhs_catalog_product_entity.entity_id')
del temp
fhs_flashsale_product_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3930772 entries, 0 to 3930771
Data columns (total 12 columns):
 #   Column                                       Dtype         
---  ------                                       -----         
 0   fhs_flashsale_product.period_id              int64         
 1   fhs_flashsale_product.product_id             int64         
 2   fhs_flashsale_product.original_price         float64       
 3   fhs_flashsale_product.flashsale_price        float64       
 4   fhs_flashsale_product.total_items            float64       
 5   fhs_flashsale_period.id                      int64         
 6   fhs_flashsale_period.start_date              datetime64[ns]
 7   fhs_flashsale_period.end_date                datetime64[ns]
 8   fhs_catalog_product_entity.entity_id         int64         
 9   fhs_catalog_product_entity.sku               object        
 10  fhs_catalog_product_entity.created_at        datetime64[ns]
 11  fhs_catalog_product_entity.discount_p

### Lọc ra những sản phẩm được đặt hàng có trong flashsale với điều kiện
- #### Ngày tạo đơn trong ngày flashsale, và sku của sản phẩm phải trùng nhau

In [10]:
# fhs_flashsale_product_order_complete = \
#           pd.merge(fhs_sales_flat_order_complete_item, fhs_flashsale_product_info)

# condition2 = np.where(
#                     (fhs_flashsale_product_info['fhs_catalog_product_entity.sku'] 
#                               == fhs_sales_flat_order_complete_item['fhs_sales_flat_order_item.sku'])
#           &         (fhs_flashsale_product_info['fhs_flashsale_period.start_date'] 
#                               >= fhs_sales_flat_order_complete_item['fhs_sales_flat_order_item.created_at'])
#           &         (fhs_flashsale_product_info['fhs_flashsale_period.end_date'] 
#                               <= fhs_sales_flat_order_complete_item['fhs_sales_flat_order_item.created_at'])
#           )
from sqlite3 import connect 
conn = connect(':memory:')
fhs_flashsale_product_info.to_sql('fhs_flashsale_product_info', conn)
fhs_sales_flat_order_complete_item.to_sql('fhs_sales_flat_order_complete_item', conn)
pd.read_sql('SELECT * \
                    FROM fhs_flashsale_product_info as A join fhs_sales_flat_order_complete_item as B \
                    on A.fhs_catalog_product_entity.sku == B.fhs_sales_flat_order_item.sku', conn)


DatabaseError: Execution failed on sql 'SELECT *                     FROM fhs_flashsale_product_info as A join fhs_sales_flat_order_complete_item as B                     on A.fhs_catalog_product_entity.sku == B.fhs_sales_flat_order_item.sku': no such column: A.fhs_catalog_product_entity.sku